In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import pandas as pd
from dotenv import load_dotenv, find_dotenv
from sqlalchemy import create_engine, MetaData, Table
import psycopg2

In [3]:
# подгружаем .env
load_dotenv()

True

In [4]:
import os

import pandas as pd
import mlflow
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import (
    OneHotEncoder, 
    SplineTransformer, 
    QuantileTransformer, 
    RobustScaler,
    PolynomialFeatures,
    KBinsDiscretizer,
)

TABLE_NAME = 'clean_users_churn'

TRACKING_SERVER_HOST = "127.0.0.1"
TRACKING_SERVER_PORT = 5000

EXPERIMENT_NAME = 'churn_denis_putov_www'
RUN_NAME = "preprocessing" 
REGISTRY_MODEL_NAME = 'model_222registry'

/home/mle-user/.venv_project_name/lib/python3.10/site-packages/pydantic/_internal/_config.py:345: UserWarning: Valid config keys have changed in V2:
* 'schema_extra' has been renamed to 'json_schema_extra'
  warnings.warn(message, UserWarning)


In [5]:
# Считываем все креды
src_host = os.environ.get('DB_SOURCE_HOST')
src_port = os.environ.get('DB_SOURCE_PORT')
src_username = os.environ.get('DB_SOURCE_USER')
src_password = os.environ.get('DB_SOURCE_PASSWORD')
src_db = os.environ.get('DB_SOURCE_NAME') 

dst_host = os.environ.get('DB_DESTINATION_HOST')
dst_port = os.environ.get('DB_DESTINATION_PORT')
dst_username = os.environ.get('DB_DESTINATION_USER')
dst_password = os.environ.get('DB_DESTINATION_PASSWORD')
dst_db = os.environ.get('DB_DESTINATION_NAME')

s3_bucket = os.environ.get('S3_BUCKET_NAME')
s3_access_key = os.environ.get('AWS_ACCESS_KEY_ID')
s3_secret_access_key = os.environ.get('AWS_SECRET_ACCESS_KEY')

In [6]:
pd.options.display.max_columns = 100
pd.options.display.max_rows = 64

In [7]:
import os

import psycopg
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import mlflow
connection = {"sslmode": "require", "target_session_attrs": "read-write"}
postgres_credentials = {
    "host": os.getenv("DB_DESTINATION_HOST"),
    "port": os.getenv("DB_DESTINATION_PORT"),
    "dbname": os.getenv("DB_DESTINATION_NAME"),
    "user": os.getenv("DB_DESTINATION_USER"),
    "password": os.getenv("DB_DESTINATION_PASSWORD"),
}

connection.update(postgres_credentials)

with psycopg.connect(**connection) as conn:

    with conn.cursor() as cur:
        cur.execute(f"SELECT * FROM {TABLE_NAME}")
        data = cur.fetchall()
        columns = [col[0] for col in cur.description]

df = pd.DataFrame(data, columns=columns)

df.head(5)

,id,customer_id,begin_date,end_date,type,paperless_billing,payment_method,monthly_charges,total_charges,internet_service,online_security,online_backup,device_protection,tech_support,streaming_tv,streaming_movies,gender,senior_citizen,partner,dependents,multiple_lines,target
0,1,7590-VHVEG,2020-01-01,NaT,Month-to-month,Yes,Electronic check,29.85,29.85,DSL,No,Yes,No,No,No,No,Female,0,Yes,No,No,0
1,2,5575-GNVDE,2017-04-01,NaT,One year,No,Mailed check,56.95,1889.50,DSL,Yes,No,Yes,No,No,No,Male,0,No,No,No,0
2,3,3668-QPYBK,2019-10-01,2019-12-01,Month-to-month,Yes,Mailed check,53.85,108.15,DSL,Yes,Yes,No,No,No,No,Male,0,No,No,No,1
3,4,7795-CFOCW,2016-05-01,NaT,One year,No,Bank transfer (automatic),42.30,1840.75,DSL,Yes,No,Yes,Yes,No,No,Male,0,No,No,No,0
4,5,9237-HQITU,2019-09-01,2019-11-01,Month-to-month,Yes,Electronic check,70.70,151.65,Fiber optic,No,No,No,No,No,No,Female,0,No,No,No,1


In [8]:
from sklearn.preprocessing import OneHotEncoder
obj_df = df.select_dtypes(include="object")

In [9]:
cat_columns = ["type", "payment_method", "internet_service", "gender"]
encoder_oh = OneHotEncoder(categories='auto', handle_unknown='ignore', max_categories=10, sparse_output=False, drop='first')
encoded_features = encoder_oh.fit_transform(df[cat_columns].to_numpy())
encoded_df = pd.DataFrame(encoded_features, columns=encoder_oh.get_feature_names_out())
obj_df = pd.concat([obj_df, encoded_df], axis=1)
obj_df.head(2)

,customer_id,type,paperless_billing,payment_method,internet_service,online_security,online_backup,device_protection,tech_support,streaming_tv,streaming_movies,gender,partner,dependents,multiple_lines,x0_One year,x0_Two year,x1_Credit card (automatic),x1_Electronic check,x1_Mailed check,x2_Fiber optic,x3_Male
0,7590-VHVEG,Month-to-month,Yes,Electronic check,DSL,No,Yes,No,No,No,No,Female,Yes,No,No,0.0,0.0,0.0,1.0,0.0,0.0,0.0
1,5575-GNVDE,One year,No,Mailed check,DSL,Yes,No,Yes,No,No,No,Male,No,No,No,1.0,0.0,0.0,0.0,1.0,0.0,1.0


In [10]:
num_columns = ["monthly_charges", "total_charges"]
display(df[num_columns])
display(df.dtypes)

,monthly_charges,total_charges
0,29.85,29.85
1,56.95,1889.50
2,53.85,108.15
3,42.30,1840.75
4,70.70,151.65
...,...,...
7038,84.80,1990.50
7039,103.20,7362.90
7040,29.60,346.45
7041,74.40,306.60


id                            int64
customer_id                  object
begin_date           datetime64[ns]
end_date             datetime64[ns]
type                         object
paperless_billing            object
payment_method               object
monthly_charges             float64
total_charges               float64
internet_service             object
online_security              object
online_backup                object
device_protection            object
tech_support                 object
streaming_tv                 object
streaming_movies             object
gender                       object
senior_citizen                int64
partner                      object
dependents                   object
multiple_lines               object
target                        int64
dtype: object

In [11]:
num_columns = ["monthly_charges", "total_charges"]
num_df = df.select_dtypes(include='float64')

In [12]:
num_columns = ["monthly_charges", "total_charges"]

n_knots = 3
degree_spline = 4
n_quantiles=100
degree = 3
n_bins = 5
encode = 'ordinal'
strategy = 'uniform'
subsample = None


# SplineTransformer
encoder_spl = SplineTransformer(n_knots=n_knots, degree=degree_spline)
encoded_features = encoder_spl.fit_transform(df[num_columns].to_numpy())

encoded_df = pd.DataFrame(
    encoded_features, 
    columns=encoder_spl.get_feature_names_out(num_columns)
)
num_df = pd.concat([num_df, encoded_df], axis=1)


# QuantileTransformer
encoder_q = QuantileTransformer(n_quantiles=n_quantiles)
encoded_features = encoder_q.fit_transform(df[num_columns].to_numpy())

encoded_df = pd.DataFrame(encoded_features, columns=encoder_q.get_feature_names_out(num_columns))
encoded_df.columns = [col + f"_q_{n_quantiles}" for col in num_columns]
num_df = pd.concat([num_df, encoded_df], axis=1)


# RobustScaler
encoder_rb = RobustScaler()
encoded_features = encoder_rb.fit_transform(df[num_columns].to_numpy())

encoded_df = pd.DataFrame(encoded_features, columns=encoder_rb.get_feature_names_out(num_columns))
encoded_df.columns = [col + f"_robust" for col in num_columns]
num_df = pd.concat([num_df, encoded_df], axis=1)


# PolynomialFeatures
encoder_pol = PolynomialFeatures(degree=degree)
encoded_features = encoder_pol.fit_transform(df[num_columns].to_numpy())


encoded_df = pd.DataFrame(encoded_features, columns=encoder_pol.get_feature_names_out(num_columns))
encoded_df.drop(encoded_df.columns[:1 + len(num_columns)], axis=1, inplace=True)
num_df = pd.concat([num_df, encoded_df], axis=1) #

# KBinsDiscretizer
encoder_kbd = KBinsDiscretizer(n_bins=n_bins, encode=encode, strategy=strategy, subsample=subsample)
encoded_features = encoder_kbd.fit_transform(df[num_columns].to_numpy())

encoded_df = pd.DataFrame(encoded_features, columns=encoder_kbd.get_feature_names_out(num_columns))
encoded_df.columns = [col + f"_bin" for col in num_columns]
num_df = pd.concat([num_df, encoded_df], axis=1)


num_df.head(2)

,monthly_charges,total_charges,monthly_charges_sp_0,monthly_charges_sp_1,monthly_charges_sp_2,monthly_charges_sp_3,monthly_charges_sp_4,monthly_charges_sp_5,total_charges_sp_0,total_charges_sp_1,total_charges_sp_2,total_charges_sp_3,total_charges_sp_4,total_charges_sp_5,monthly_charges_q_100,total_charges_q_100,monthly_charges_robust,total_charges_robust,monthly_charges^2,monthly_charges total_charges,total_charges^2,monthly_charges^3,monthly_charges^2 total_charges,monthly_charges total_charges^2,total_charges^3,monthly_charges_bin,total_charges_bin
0,29.85,29.85,0.014583,0.335266,0.554993,0.095040,0.000118,0.0,0.041243,0.457057,0.459607,0.042093,1.762313e-12,0.0,0.232658,0.027398,-0.74517,-0.405008,891.0225,891.0225,8.910225e+02,26597.021625,2.659702e+04,2.659702e+04,2.659702e+04,0.0,0.0
1,56.95,1889.50,0.000116,0.094742,0.554677,0.335807,0.014658,0.0,0.004345,0.230314,0.596051,0.167842,1.447607e-03,0.0,0.394114,0.580098,-0.24655,0.144473,3243.3025,107607.0250,3.570210e+06,184706.077375,6.128220e+06,2.033235e+08,6.745912e+09,1.0,1.0


In [13]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder

# Создание преобразователей
numeric_transformer = ColumnTransformer(transformers=[('spl', encoder_spl, num_columns), ('q', encoder_q, num_columns), ('rb', encoder_rb, num_columns), ('pol', encoder_pol, num_columns), ('kbd', encoder_kbd, num_columns)])

categorical_transformer = Pipeline(steps=[('encoder', encoder_oh)])

# Общий препроцессор
preprocessor = ColumnTransformer(transformers=[('num', numeric_transformer, num_columns), ('cat', categorical_transformer, cat_columns)], n_jobs=-1)

# Применение преобразований
encoded_features = preprocessor.fit_transform(df)

# Создание DataFrame с новыми названиями колонок
transformed_df = pd.DataFrame(encoded_features,columns=preprocessor.get_feature_names_out())

# Обновление исходного DataFrame
df = pd.concat([df, transformed_df], axis=1)
df.head(2)

,id,customer_id,begin_date,end_date,type,paperless_billing,payment_method,monthly_charges,total_charges,internet_service,online_security,online_backup,device_protection,tech_support,streaming_tv,streaming_movies,gender,senior_citizen,partner,dependents,multiple_lines,target,num__spl__monthly_charges_sp_0,num__spl__monthly_charges_sp_1,num__spl__monthly_charges_sp_2,num__spl__monthly_charges_sp_3,num__spl__monthly_charges_sp_4,num__spl__monthly_charges_sp_5,num__spl__total_charges_sp_0,num__spl__total_charges_sp_1,num__spl__total_charges_sp_2,num__spl__total_charges_sp_3,num__spl__total_charges_sp_4,num__spl__total_charges_sp_5,num__q__monthly_charges,num__q__total_charges,num__rb__monthly_charges,num__rb__total_charges,num__pol__1,num__pol__monthly_charges,num__pol__total_charges,num__pol__monthly_charges^2,num__pol__monthly_charges total_charges,num__pol__total_charges^2,num__pol__monthly_charges^3,num__pol__monthly_charges^2 total_charges,num__pol__monthly_charges total_charges^2,num__pol__total_charges^3,num__kbd__monthly_charges,num__kbd__total_charges,cat__type_One year,cat__type_Two year,cat__payment_method_Credit card (automatic),cat__payment_method_Electronic check,cat__payment_method_Mailed check,cat__internet_service_Fiber optic,cat__gender_Male
0,1,7590-VHVEG,2020-01-01,NaT,Month-to-month,Yes,Electronic check,29.85,29.85,DSL,No,Yes,No,No,No,No,Female,0,Yes,No,No,0,0.014583,0.335266,0.554993,0.095040,0.000118,0.0,0.041243,0.457057,0.459607,0.042093,1.762313e-12,0.0,0.232658,0.027398,-0.74517,-0.405008,1.0,29.85,29.85,891.0225,891.0225,8.910225e+02,26597.021625,2.659702e+04,2.659702e+04,2.659702e+04,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
1,2,5575-GNVDE,2017-04-01,NaT,One year,No,Mailed check,56.95,1889.50,DSL,Yes,No,Yes,No,No,No,Male,0,No,No,No,0,0.000116,0.094742,0.554677,0.335807,0.014658,0.0,0.004345,0.230314,0.596051,0.167842,1.447607e-03,0.0,0.394114,0.580098,-0.24655,0.144473,1.0,56.95,1889.50,3243.3025,107607.0250,3.570210e+06,184706.077375,6.128220e+06,2.033235e+08,6.745912e+09,1.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0


In [14]:
os.environ.update({
    "MLFLOW_S3_ENDPOINT_URL": "https://storage.yandexcloud.net",
    "AWS_ACCESS_KEY_ID": os.getenv("AWS_ACCESS_KEY_ID"),
    "AWS_SECRET_ACCESS_KEY": os.getenv("AWS_SECRET_ACCESS_KEY"),
})

mlflow.set_tracking_uri(f"http://{TRACKING_SERVER_HOST}:{TRACKING_SERVER_PORT}")
mlflow.set_registry_uri(f"http://{TRACKING_SERVER_HOST}:{TRACKING_SERVER_PORT}")

experiment_id = mlflow.get_experiment_by_name(EXPERIMENT_NAME).experiment_id

with mlflow.start_run(run_name=RUN_NAME, experiment_id=experiment_id) as run:
    run_id = run.info.run_id

    mlflow.sklearn.log_model(preprocessor, "column_transformer") 

2025/03/28 10:43:08 WARNING mlflow.sklearn: Model was missing function: predict. Not logging python_function flavor!


In [15]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, f1_score
import mlflow


In [16]:
X = transformed_df
y = df['target']  # Замените 'target' на имя вашей целевой переменной
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 2. Инициализация и обучение модели
model = RandomForestClassifier(
    n_estimators=100,
    max_depth=10,
    random_state=42
)

In [19]:
# 3. Запуск эксперимента MLflow
mlflow.set_tracking_uri(f"http://{TRACKING_SERVER_HOST}:{TRACKING_SERVER_PORT}")
mlflow.set_registry_uri(f"http://{TRACKING_SERVER_HOST}:{TRACKING_SERVER_PORT}")

experiment_id = mlflow.get_experiment_by_name(EXPERIMENT_NAME).experiment_id

mlflow.set_experiment(EXPERIMENT_NAME)

<Experiment: artifact_location='s3://s3-student-mle-20250130-b8bcb9fce1/15', creation_time=1742715896635, experiment_id='15', last_update_time=1742715896635, lifecycle_stage='active', name='churn_denis_putov_www', tags={}>

In [20]:
with mlflow.start_run():
    # Обучение модели
    model.fit(X_train, y_train)
    
    # Прогнозирование и оценка
    y_pred = model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    
    # Логирование параметров
    mlflow.log_params({
        "model_type": "RandomForest",
        "n_estimators": 100,
        "max_depth": 10
    })
    
    # Логирование метрик
    mlflow.log_metrics({
        "accuracy": accuracy,
        "f1_score": f1
    })
    
    # Логирование артефактов
    mlflow.sklearn.log_model(
        sk_model=model,
        artifact_path="model",
        registered_model_name="Churn_Model_v2"
    )
    
    # Сравнение с предыдущей версией
    print(f"Новые метрики: Accuracy={accuracy:.4f}, F1={f1:.4f}")

# 4. Сравнение с базовой моделью (пример)
# Замените значения на метрики вашей предыдущей модели
baseline_accuracy = 0.82  
baseline_f1 = 0.78

print(f"\nИзменение качества:")
print(f"Accuracy: {baseline_accuracy} -> {accuracy:.4f} ({accuracy - baseline_accuracy:+.4f})")
print(f"F1-score: {baseline_f1} -> {f1:.4f} ({f1 - baseline_f1:+.4f})")

Successfully registered model 'Churn_Model_v2'.
2025/03/28 11:03:27 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation. Model name: Churn_Model_v2, version 1


Новые метрики: Accuracy=0.8126, F1=0.5756

Изменение качества:
Accuracy: 0.82 -> 0.8126 (-0.0074)
F1-score: 0.78 -> 0.5756 (-0.2044)


Created version '1' of model 'Churn_Model_v2'.
